In [36]:
from sage.interfaces.gap import  get_gap_memory_pool_size, set_gap_memory_pool_size
set_gap_memory_pool_size(24364842180)



W = WeylGroup("E6", prefix="s")
[s1,s2,s3,s4,s5,s6] = W.simple_reflections()

##################################################################################

n = rank(W)
w0 = W.long_element()
e = W(1)


####### Kazhdan-Lusztig polynomials ##########################

# A faster implementation of KL-polynomials (using the optional package Coxeter 3) is given by this
# Fokko Ducloux’s Coxeter3 C++ library.

# Had to install it: I just typed "sage -i coxeter3" in the terminal.

# It seems that one can direcly coerce from WeylGroup to CoxeterGroup and vice versa.
# I will therefore use CoxeterGroup to calculate KL-polynomials, but for all other Bruhat business I will use WeylGroup.

R.<q> = LaurentPolynomialRing(QQ)

KL = KazhdanLusztigPolynomial(W,q)  # KL-polynomials implemented in standard Sage way
# http://doc.sagemath.org/html/en/reference/combinat/sage/combinat/kazhdan_lusztig.html


CoxeterPackage = CoxeterGroup(W, implementation="coxeter3")

def KLP(x,y):
    '''Returns the KL-polynomial, implemented in "Coxeter3" package by Fokko du Cloux.
    http://math.univ-lyon1.fr/~ducloux/coxeter/coxeter3/english/coxeter3_e.html'''
    
    if x not in W:
        x = convert_from_123(x)
    if y not in W:
        y = convert_from_123(y)
    
    return CoxeterPackage.kazhdan_lusztig_polynomial(CoxeterPackage(x), CoxeterPackage(y))
    # If "coxeter3" is not installed, remove the line 'CoxeterPackage = CoxeterGroup(W, implementation="coxeter3")'
    # and in this function return KL.P(x,y)

#Point:
#    - standard Sage way: KL.P(x,y)
#    - faster way: KLP(x,y) 


def mu(w,x):
    '''Returns the KL mu-function with arguments w,x.
    By Humphrey's BGG book p. 175 and p. 169, for w<x we have:
    mu(x,w) = mu(w0*w,w0*x),
    mu(w,x) = dim Exit^1 (L_w,L_x) = dim Exit^1(L_x,L_w) = dim Exit^1(Delta_x,L_w).'''

    if w not in W:
        w = convert_from_123(w)
    if x not in W:
        x = convert_from_123(x)
        
    if w.bruhat_le(x):
        poly_dict = KLP(w,x).dict()       
        j = (x.length()-w.length()-1)/2 
        if j not in poly_dict.keys():
            return 0
        return poly_dict[j]

    return 0


def convert_to_123(w):
    '''Converts an element from W to the "123" string notation.
    Does not work with coefficients, as "convert_to_123_long".'''
    
    if w == W(1):
        return "e"
    
    return str(w).replace("s","").replace("*","")


def convert_from_123(string):
    '''Converts one element from W in the "123" string notation to the usual "s1*s2*s3" notation.'''
    
    if type(string)== Integer:
        string = str(string)
        
    if string == "e":
        return W(1)
    
    string = "*".join([char for char in string])
    
    for i in range(1,n+1):
        string = string.replace(str(i),"s%s"%i)
    
    return eval(string)


def DR(w):
    '''Returns the set of simple right descents of w.'''
    
    return {W.simple_reflections()[i] for i in w.descents()}


def DL(w):
    '''Returns the set of simple left   descents of w.'''
        
    return DR(w.inverse())


def AL(w):
    '''Returns the set of simple left ascends of w.'''
    
    DescLe = list(DL(w))
    AscLe = [s for s in W.simple_reflections() if s not in DescLe]
    return set(AscLe)


def AR(w):
    '''Returns the set of simple right ascends of w.'''
    
    DescRi = list(DR(w))
    AscRi = [s for s in W.simple_reflections() if s not in DescRi]
    return set(AscRi)



# W_poset = W.bruhat_poset()

def W_subposet(L):
    '''This is equivalent to W.bruhat_poset().subposet(),
    but without calculating the full W.bruhat_poset().'''
    
    def br_le(x,y):
        return x.bruhat_le(y)    
    return Poset((L,br_le))




def join(S):
    SS = [convert_from_123(a) for a in S if a not in W] + [a for a in S if a in W]
    
    U = set(W.bruhat_interval(SS[0],w0))
    for a in SS[1:]:
        U = U.intersection(set(W.bruhat_interval(a,w0)))
        
    minU = (W_subposet(list(U))).minimal_elements()
    
    if len(minU)==1:
        j = minU[0]
        return eval(convert_to_123(str(j)))
    else:
        return minU

    
def join_B(S):
    '''Here should S consists of join-irreds with fixed descents.'''
    SS = [convert_from_123(a) for a in S if a not in W] + [a for a in S if a in W]
    bigrassmannians_fixed = [x for x in bigrassmannians if DL(x)==DL(SS[0]) and DR(x)==DR(SS[0])]
    
    U = set(x for x in bigrassmannians_fixed if (SS[0]).bruhat_le(x) )
    for a in SS[1:]:
        U = U.intersection(set(x for x in bigrassmannians_fixed if a.bruhat_le(x) ))
        
    minU = (W_subposet(list(U))).minimal_elements()
    
    if len(minU)==1:
        j = minU[0]
        return eval(convert_to_123(str(j)))
    else:
        return minU
    
    
def JM(w):
    if w not in W:
        w = convert_from_123(w)
    result = W_poset.subposet([x for x in join_irreducibles if x.bruhat_le(w) ]).maximal_elements()
    return [convert_from_123(convert_to_123(x)) for x in result]

def BM(w):
    if w not in W:
        w = convert_from_123(w)
    return W_poset.subposet([x for x in bigrassmannians if x.bruhat_le(w) ]).maximal_elements()

def JM2(w):
    if w not in W:
        w = convert_from_123(w)
    DLw = DL(w)
    DRw = DR(w)
    result = []
    for x in JM(w):
        x = convert_from_123(convert_to_123(x))
        if list(DL(x))[0] in DLw and list(DR(x))[0] in DRw:
            result.append(x)
    return result









In [24]:
######################## D4 ########################

#sum_J KL = 28

#bigrassmannians: 30

bigrassmannians = [s4*s2*s3*s1*s2*s4, s2*s4*s1*s2, s2*s3*s1*s2, s2*s4*s3*s2, s2*s1, s2*s4, s2*s3, s1*s2, s4*s2, s1*s2*s4*s3*s2*s1, s3*s2*s4*s1*s2*s3, s3*s2, s1, s4, s2*s4*s1*s2*s3, s1*s2*s4*s3*s2, s2*s3*s1*s2*s4*s2*s3*s1*s2, s4*s2*s1, s1*s2*s4, s1*s2*s3, s2, s3, s3*s2*s4*s1*s2, s4*s2*s3*s1*s2, s2*s4*s3*s2*s1, s2*s3*s1*s2*s4, s3*s2*s1, s3*s2*s4, s4*s2*s3, s2*s4*s3*s1*s2]

#join_irreducibles: 29

join_irreducibles = [s3, s4, s2, s1, s2*s4*s3*s2, s3*s2*s1, s2*s4*s1*s2, s1*s2*s3, s4*s2*s1, s1*s2*s4, s3*s2*s4, s1*s2, s2*s4, s3*s2*s4*s1*s2*s3, s1*s2*s4*s3*s2*s1, s4*s2*s3, s3*s2, s4*s2, s4*s2*s3*s1*s2*s4, s1*s2*s4*s3*s2, s2*s1, s4*s2*s3*s1*s2, s3*s2*s4*s1*s2, s2*s4*s1*s2*s3, s2*s3*s1*s2, s2*s3*s1*s2*s4, s2*s4*s3*s2*s1, s2*s3, s2*s3*s1*s2*s4*s2*s3*s1*s2]

#dissectors: 26

dissectors = [s4*s2*s3*s1*s2*s4, s2*s1, s2*s4, s2*s3, s1*s2, s4*s2, s1*s2*s4*s3*s2*s1, s3*s2*s4*s1*s2*s3, s3*s2, s1, s4, s2*s4*s1*s2*s3, s1*s2*s4*s3*s2, s2*s3*s1*s2*s4*s2*s3*s1*s2, s4*s2*s1, s1*s2*s4, s1*s2*s3, s2, s3, s3*s2*s4*s1*s2, s4*s2*s3*s1*s2, s2*s4*s3*s2*s1, s2*s3*s1*s2*s4, s3*s2*s1, s3*s2*s4, s4*s2*s3]


In [19]:
######################## B4 ########################

#sum_J KL = 44

#bigrassmannians: 45

bigrassmannians = [s2*s3*s1*s2, s4*s3*s4*s2*s3*s4*s1*s2*s3*s4, s2*s1, s2*s3, s1*s2, s3*s2, s3*s4*s2*s3*s4*s1*s2*s3, s1, s3, s1*s2*s3, s3*s2*s1, s3*s4*s2*s3*s4*s2*s3*s1*s2, s3*s4*s2*s3*s4, s2, s3*s4*s2*s3*s4*s2*s3, s2*s3*s4*s1*s2*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s4, s4*s3*s4, s4*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s4*s2*s3, s4*s3*s4*s2*s3, s1*s2*s3*s4*s3*s2*s1, s4*s3*s4*s2*s3*s4*s1*s2*s3, s3*s4*s3, s3*s4*s3*s2*s1, s1*s2*s3*s4*s3, s2*s3*s4*s3*s2, s4, s3*s4*s2*s3*s4*s1*s2*s3*s4, s2*s3*s4, s4*s3*s2, s1*s2*s3*s4*s3*s2, s4*s3, s4*s3*s4*s2*s3*s4, s1*s2*s3*s4, s3*s4*s2*s3*s4*s1*s2*s3*s4*s1*s2*s3, s4*s3*s2*s1, s2*s3*s4*s3*s2*s1, s2*s3*s4*s1*s2*s3, s3*s4, s3*s4*s2*s3*s1*s2, s2*s3*s4*s3*s1*s2, s3*s4*s2*s3, s2*s3*s4*s3, s3*s4*s3*s2]

#join_irreducibles = dissectors: 44

join_irreducibles = [s4, s3, s2, s1, s3*s2*s1, s1*s2*s3, s2*s3*s4*s3*s2, s3*s4*s2*s3*s4*s1*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s3*s2*s1, s3*s4*s3*s2*s1, s1*s2*s3*s4*s3, s4*s3*s4*s2*s3*s4, s2*s3*s1*s2, s3*s4*s2*s3, s4*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s4, s3*s4*s3, s4*s3*s4*s2*s3, s2*s1, s3*s4*s2*s3*s4, s1*s2, s2*s3*s4*s3, s4*s3*s2*s1, s3*s4*s3*s2, s1*s2*s3*s4, s2*s3*s4*s3*s2*s1, s3*s4*s2*s3*s4*s2*s3*s1*s2, s3*s2, s2*s3*s4*s1*s2*s3*s4*s2*s3, s2*s3, s1*s2*s3*s4*s3*s2, s2*s3*s4*s1*s2*s3, s4*s3*s2, s3*s4*s2*s3*s4*s1*s2*s3, s2*s3*s4, s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s4*s2*s3*s1*s2, s4*s3*s4, s3*s4*s2*s3*s4*s2*s3, s4*s3, s3*s4*s2*s3*s4*s1*s2*s3*s4, s4*s3*s4*s2*s3*s4*s1*s2*s3*s4, s3*s4, s4*s3*s4*s2*s3*s4*s1*s2*s3]

dissectors = [x for x in join_irreducibles]

In [ ]:
######################## F4 ########################

#sum_J KL = 92

#bigrassmannians: 108

bigrassmannians = [s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s3*s2*s3, s4*s3*s2*s3*s4, s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4, s4*s3*s2*s3*s4*s1*s2*s3, s4*s3*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s4*s1*s2*s3*s2*s1, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3, s3*s4, s3*s2, s3*s4*s2*s3, s3*s4*s2*s3*s2, s3*s4*s2*s3*s2*s1, s3*s2*s1, s3*s4*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4, s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3, s3*s2*s3*s4, s3*s2*s3*s4*s3*s2*s3, s3*s2*s3*s4*s3*s2*s3*s1*s2, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s4*s3*s1*s2*s3, s3*s2*s3*s4*s3*s1*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3*s4*s3*s1*s2*s3, s3*s2*s3*s1*s2, s3*s2*s3*s4*s1*s2*s3, s3*s2*s3*s4*s1*s2*s3*s2, s3*s2*s3*s4*s1*s2*s3*s2*s1, s3*s4*s2*s3*s1*s2*s3*s4*s3*s1*s2*s3*s1*s2, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s4*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2, s2*s3, s2*s3*s4, s2*s3*s2, s2*s3*s4*s2*s3, s2*s3*s4*s2*s3*s2, s2*s3*s4*s2*s3*s2*s1, s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3*s4, s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s4*s1*s2*s3*s2, s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s4*s1*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s1*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s1*s2*s3, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s2, s1*s2*s3*s4*s2*s3, s1*s2*s3*s4*s2*s3*s2, s1*s2*s3*s4*s2*s3*s2*s1, s1*s2*s3*s2*s1, s1*s2*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4, s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1]

#join_irreducibles: 96

join_irreducibles = [s3, s2, s1, s4, s3*s4, s2*s3, s1*s2, s2*s3*s2, s3*s4*s2*s3, s2*s3*s2*s1, s1*s2*s3, s1*s2*s3*s2, s1*s2*s3*s4, s3*s4*s2*s3*s2, s3*s2*s3*s4*s3*s2*s3, s3*s4*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3, s2*s3*s1*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s2, s3*s4*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3*s2*s1, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s4*s3, s2*s3*s4, s2*s3*s4*s2*s3, s2*s3*s4*s2*s3*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4, s4*s3*s2, s4*s3*s2*s3, s1*s2*s3*s2*s1, s2*s3*s4*s2*s3*s2*s1, s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s4*s3*s2*s1, s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s4*s3*s2*s3*s4, s4*s3*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s4*s3*s2*s3*s4*s1*s2*s3*s2*s1, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2, s3*s2*s3, s3*s4*s2*s3*s2*s1, s3*s2*s3*s4*s3*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s4, s3*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s3*s2*s3*s4*s1*s2*s3*s2, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s3*s2*s1, s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2*s1, s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4, s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2*s3*s4*s1*s2*s3*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2]

#dissectors: 80

dissectors = [s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s3*s2*s3, s4*s3*s2*s3*s4, s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4, s4*s3*s2*s3*s4*s1*s2*s3, s4*s3*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s4*s1*s2*s3*s2*s1, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3, s3*s4, s3*s2, s3*s4*s2*s3, s3*s4*s2*s3*s2, s3*s4*s2*s3*s2*s1, s3*s2*s1, s3*s4*s2*s3*s1*s2*s3*s4, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3, s3*s2*s3*s4, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s1*s2, s3*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2, s2*s3, s2*s3*s4, s2*s3*s2, s2*s3*s4*s2*s3, s2*s3*s4*s2*s3*s2, s2*s3*s4*s2*s3*s2*s1, s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2*s3*s4, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s2, s1*s2*s3*s4*s2*s3, s1*s2*s3*s4*s2*s3*s2, s1*s2*s3*s4*s2*s3*s2*s1, s1*s2*s3*s2*s1, s1*s2*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4, s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1]


In [30]:
######################## D5 ########################

#sum_J KL = 60

#bigrassmannians: 69

bigrassmannians = [s3, s3*s2, s3*s4, s3*s5, s3*s2*s1, s3*s4*s2*s3, s3*s5*s2*s3, s3*s5*s4*s3, s3*s4*s2*s3*s1*s2, s3*s5*s2*s3*s1*s2, s2, s2*s3, s2*s3*s4, s2*s3*s5, s2*s3*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s5*s1*s2*s3, s4, s4*s3, s4*s3*s2, s4*s3*s5, s4*s3*s2*s1, s4*s3*s5*s2*s3, s4*s3*s5*s2*s3*s1*s2, s5, s5*s3, s5*s3*s2, s5*s3*s4, s5*s3*s2*s1, s5*s3*s4*s2*s3, s5*s3*s4*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s5, s1*s2*s3*s5*s4*s3, s1*s2*s3*s5*s4*s3*s2, s1*s2*s3*s5*s4*s3*s2*s1, s5*s3*s4*s2*s3*s5, s5*s3*s4*s2*s3*s5*s1*s2*s3, s4*s3*s5*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3, s2*s3*s5*s4*s3*s2, s3*s4*s2*s3*s5*s3*s4*s2*s3, s3*s4*s2*s3*s5, s3*s5*s4*s2*s3, s3*s5*s2*s3*s4, s3*s5*s4*s3*s2, s3*s5*s4*s3*s2*s1, s3*s5*s4*s2*s3*s1*s2, s3*s4*s2*s3*s5*s1*s2*s3, s3*s5*s2*s3*s4*s1*s2*s3, s5*s3*s4*s2*s3*s5*s1*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s4*s3*s2*s1, s3*s4*s2*s3*s5*s3*s4*s2*s3*s1*s2, s3*s4*s2*s3*s5*s3*s4*s1*s2*s3, s3*s5*s2*s3*s4*s1*s2*s3*s5*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s5*s1*s2*s3*s4, s3*s5*s2*s3*s4*s1*s2*s3*s5, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3, s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s4*s1*s2*s3, s2*s3*s5*s1*s2*s3*s4, s2*s3*s5*s4*s3*s1*s2]

#join_irreducibles: 64

join_irreducibles = [s3, s2, s4, s5, s1, s2*s3, s4*s3, s5*s3, s1*s2, s2*s3*s4, s4*s3*s2, s2*s3*s5, s5*s3*s2, s4*s3*s5, s5*s3*s4, s4*s3*s2*s1, s1*s2*s3, s5*s3*s2*s1, s3*s2, s3*s2*s1, s1*s2*s3*s4, s1*s2*s3*s5, s3*s4, s3*s4*s2*s3, s3*s4*s2*s3*s1*s2, s3*s5, s3*s5*s2*s3, s3*s5*s2*s3*s1*s2, s3*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s5*s1*s2*s3, s3*s4*s2*s3*s5*s3*s4*s2*s3, s3*s5*s4*s3*s2, s3*s5*s2*s3*s4, s3*s4*s2*s3*s5, s2*s3*s5*s4*s3*s2, s2*s3*s5*s4*s3, s3*s5*s4*s3*s2*s1, s4*s3*s5*s2*s3*s4, s4*s3*s5*s2*s3, s4*s3*s5*s2*s3*s1*s2, s5*s3*s4*s2*s3*s5, s5*s3*s4*s2*s3, s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s1*s2*s3*s4, s3*s4*s2*s3*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3, s3*s5*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s5*s1*s2*s3, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3*s1*s2, s2*s3*s5*s4*s3*s2*s1, s1*s2*s3*s5*s4*s3, s1*s2*s3*s5*s4*s3*s2, s1*s2*s3*s5*s4*s3*s2*s1, s5*s3*s4*s2*s3*s5*s1*s2*s3, s4*s3*s5*s2*s3*s4*s1*s2*s3, s5*s3*s4*s2*s3*s5*s1*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3*s5, s3*s5*s2*s3*s4*s1*s2*s3*s5*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s5*s1*s2*s3*s4, s3*s5*s2*s3*s4*s1*s2*s3*s5]


#dissectors: 52

dissectors = [s3, s3*s2, s3*s4, s3*s5, s3*s2*s1, s2, s2*s3, s2*s3*s4, s2*s3*s5, s2*s1, s2*s3*s1*s2, s4, s4*s3, s4*s3*s2, s4*s3*s5, s4*s3*s2*s1, s4*s3*s5*s2*s3, s4*s3*s5*s2*s3*s1*s2, s5, s5*s3, s5*s3*s2, s5*s3*s4, s5*s3*s2*s1, s5*s3*s4*s2*s3, s5*s3*s4*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s5, s1*s2*s3*s5*s4*s3, s1*s2*s3*s5*s4*s3*s2, s1*s2*s3*s5*s4*s3*s2*s1, s5*s3*s4*s2*s3*s5, s5*s3*s4*s2*s3*s5*s1*s2*s3, s4*s3*s5*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3, s2*s3*s5*s4*s3*s2, s3*s4*s2*s3*s5, s3*s5*s2*s3*s4, s3*s5*s4*s3*s2*s1, s5*s3*s4*s2*s3*s5*s1*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s4*s3*s2*s1, s3*s4*s2*s3*s5*s3*s4*s2*s3*s1*s2, s3*s5*s2*s3*s4*s1*s2*s3*s5*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s5*s1*s2*s3*s4, s3*s5*s2*s3*s4*s1*s2*s3*s5, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3, s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s1*s2*s3*s4]


In [ ]:
######################## B5 ########################

#sum_J KL = 85

#bigrassmannians: 90

bigrassmannians = [s3, s3*s2, s3*s4, s3*s2*s1, s3*s4*s5, s3*s4*s2*s3, s3*s4*s5*s4, s3*s4*s5*s4*s3, s3*s4*s2*s3*s1*s2, s2, s2*s3, s2*s3*s4, s2*s3*s4*s5, s2*s3*s4*s5*s4, s2*s3*s4*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s5, s4*s5*s3*s4, s4*s5*s3*s4*s5, s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s4, s4*s5*s4*s3, s4*s5*s4*s3*s2, s4*s5*s4*s3*s2*s1, s4*s5*s3*s4*s5*s3*s4, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s4*s5, s1*s2*s3*s4*s5*s4, s1*s2*s3*s4*s5*s4*s3, s5, s5*s4, s5*s4*s3, s5*s4*s3*s2, s5*s4*s3*s2*s1, s5*s4*s5, s5*s4*s5*s3*s4, s5*s4*s5*s3*s4*s5, s5*s4*s5*s3*s4*s2*s3, s5*s4*s5*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s5*s4*s3*s2, s1*s2*s3*s4*s5*s4*s3*s2*s1, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5, s5*s4*s5*s3*s4*s5*s2*s3*s4, s5*s4*s5*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4, s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s5*s4*s1*s2*s3, s2*s3*s4*s5*s4*s3*s1*s2, s4*s5*s3*s4*s5*s2*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s3*s4*s5*s2*s3*s4*s5, s4*s5*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s4*s3*s2, s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3, s3*s4*s5*s2*s3*s4*s5*s3*s4, s3*s4*s5*s2*s3*s4*s5, s3*s4*s5*s2*s3*s4, s3*s4*s5*s4*s2*s3, s3*s4*s5*s4*s3*s2, s3*s4*s5*s4*s3*s2*s1, s3*s4*s5*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s5*s2*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s4*s3*s2*s1, s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4*s5*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s2*s3*s4, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5]

#join_irreducibles = dissectors: 85

join_irreducibles = [s3, s3*s2, s3*s4, s3*s2*s1, s3*s4*s5, s3*s4*s2*s3, s3*s4*s5*s4, s3*s4*s5*s4*s3, s3*s4*s2*s3*s1*s2, s2, s2*s3, s2*s3*s4, s2*s3*s4*s5, s2*s3*s4*s5*s4, s2*s3*s4*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s5, s4*s5*s3*s4, s4*s5*s3*s4*s5, s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s4, s4*s5*s4*s3, s4*s5*s4*s3*s2, s4*s5*s4*s3*s2*s1, s4*s5*s3*s4*s5*s3*s4, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s4*s5, s1*s2*s3*s4*s5*s4, s1*s2*s3*s4*s5*s4*s3, s5, s5*s4, s5*s4*s3, s5*s4*s3*s2, s5*s4*s3*s2*s1, s5*s4*s5, s5*s4*s5*s3*s4, s5*s4*s5*s3*s4*s5, s5*s4*s5*s3*s4*s2*s3, s5*s4*s5*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s5*s4*s3*s2, s1*s2*s3*s4*s5*s4*s3*s2*s1, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5, s5*s4*s5*s3*s4*s5*s2*s3*s4, s5*s4*s5*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4, s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s3*s4*s5*s2*s3*s4*s5, s4*s5*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s4*s3*s2, s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3, s3*s4*s5*s2*s3*s4*s5*s3*s4, s3*s4*s5*s2*s3*s4*s5, s3*s4*s5*s2*s3*s4, s3*s4*s5*s4*s3*s2, s3*s4*s5*s4*s3*s2*s1, s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s5*s2*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s4*s3*s2*s1, s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s2*s3*s4, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5]

dissectors = [x for x in join_irreducibles]

In [ ]:
######################## D6 ########################

#sum_J KL = 110

#bigrassmannians: 135

bigrassmannians = [s3, s3*s2, s3*s4, s3*s2*s1, s3*s4*s5, s3*s4*s6, s3*s4*s2*s3, s3*s4*s6*s5*s4, s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4, s3*s4*s6*s2*s3*s4, s3*s4*s6*s5*s4*s3, s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s6*s2*s3*s4*s1*s2*s3, s2, s2*s3, s2*s3*s4, s2*s3*s4*s5, s2*s3*s4*s6, s2*s3*s4*s6*s5*s4, s2*s3*s4*s6*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s6*s1*s2*s3*s4, s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s5, s4*s5*s3*s4, s4*s5*s3*s4*s6, s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s3*s4*s6*s2*s3*s4, s4*s5*s3*s4*s6*s2*s3*s4*s1*s2*s3, s4*s6, s4*s6*s3*s4, s4*s6*s3*s4*s5, s4*s6*s3*s4*s2*s3, s4*s6*s3*s4*s2*s3*s1*s2, s4*s6*s3*s4*s5*s2*s3*s4, s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s6*s5*s3*s4, s4*s6*s5*s3*s4*s2*s3, s4*s6*s5*s3*s4*s2*s3*s1*s2, s4*s6*s5*s4, s4*s6*s5*s4*s3, s4*s6*s5*s4*s3*s2, s4*s6*s5*s4*s3*s2*s1, s4*s5*s3*s4*s6*s4*s5*s3*s4, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s4*s5, s1*s2*s3*s4*s6, s1*s2*s3*s4*s6*s5*s4, s1*s2*s3*s4*s6*s5*s4*s3, s5, s5*s4, s5*s4*s3, s5*s4*s3*s2, s5*s4*s3*s2*s1, s5*s4*s6, s5*s4*s6*s3*s4, s5*s4*s6*s3*s4*s5, s5*s4*s6*s3*s4*s2*s3, s5*s4*s6*s3*s4*s2*s3*s1*s2, s5*s4*s6*s3*s4*s5*s2*s3*s4, s5*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s6, s6*s4, s6*s4*s3, s6*s4*s3*s2, s6*s4*s3*s2*s1, s6*s4*s5, s6*s4*s5*s3*s4, s6*s4*s5*s3*s4*s6, s6*s4*s5*s3*s4*s2*s3, s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s4*s5*s3*s4*s6*s2*s3*s4, s6*s4*s5*s3*s4*s6*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s6*s5*s4*s3*s2*s1, s1*s2*s3*s4*s6*s5*s4*s3*s2, s5*s4*s6*s3*s4*s5*s2*s3*s4*s6, s5*s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4, s6*s4*s5*s3*s4*s6*s2*s3*s4*s5, s6*s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5, s6*s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6, s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4, s2*s3*s4*s5*s1*s2*s3*s4*s6, s2*s3*s4*s6*s5*s1*s2*s3*s4, s2*s3*s4*s6*s1*s2*s3*s4*s5, s2*s3*s4*s6*s5*s4*s1*s2*s3, s2*s3*s4*s6*s5*s4*s3*s1*s2, s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s6*s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s3*s4*s6*s4*s5*s2*s3*s4, s4*s5*s3*s4*s6*s4*s5*s2*s3*s4*s1*s2*s3, s4*s6*s3*s4*s5*s2*s3*s4*s6, s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4, s4*s5*s3*s4*s6*s2*s3*s4*s5, s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s6*s5*s4*s3*s2, s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3, s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4, s3*s4*s5*s2*s3*s4*s6, s3*s4*s6*s5*s2*s3*s4, s3*s4*s6*s2*s3*s4*s5, s3*s4*s6*s5*s4*s2*s3, s3*s4*s6*s5*s4*s3*s2, s3*s4*s6*s5*s4*s3*s2*s1, s3*s4*s6*s5*s4*s2*s3*s1*s2, s3*s4*s6*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4, s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s1*s2*s3*s4, s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5, s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6, s2*s3*s4*s6*s5*s4*s3*s2*s1, s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s6*s4*s5*s1*s2*s3*s4, s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4, s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5, s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6]

#join_irreducibles: 120

join_irreducibles = [s6,  s5,  s4,  s3,  s2,  s1,  s6*s4,  s5*s4,  s4*s6,  s4*s5,  s4*s3,  s3*s4,  s3*s2,  s2*s3,  s2*s1,  s1*s2,  s6*s4*s5,  s6*s4*s3,  s5*s4*s6,  s5*s4*s3,  s4*s3*s2,  s3*s4*s6,  s3*s4*s5,  s3*s2*s1,  s2*s3*s4,  s1*s2*s3,  s6*s4*s3*s2,  s5*s4*s3*s2,  s4*s6*s5*s4,  s4*s6*s3*s4,  s4*s5*s3*s4,  s4*s3*s2*s1,  s3*s4*s2*s3,  s2*s3*s4*s6,  s2*s3*s4*s5,  s2*s3*s1*s2,  s1*s2*s3*s4,  s6*s4*s5*s3*s4,  s6*s4*s3*s2*s1,  s5*s4*s6*s3*s4,  s5*s4*s3*s2*s1,  s4*s5*s3*s4*s6,  s4*s6*s3*s4*s5,  s4*s6*s5*s4*s3,  s3*s4*s6*s5*s4,  s1*s2*s3*s4*s6,  s1*s2*s3*s4*s5,  s6*s4*s5*s3*s4*s6,  s5*s4*s6*s3*s4*s5,  s4*s6*s3*s4*s2*s3,  s4*s5*s3*s4*s2*s3,  s4*s6*s5*s4*s3*s2,  s3*s4*s6*s2*s3*s4,  s3*s4*s5*s2*s3*s4,  s3*s4*s6*s5*s4*s3,  s3*s4*s2*s3*s1*s2,  s2*s3*s4*s6*s5*s4,  s2*s3*s4*s1*s2*s3,  s6*s4*s5*s3*s4*s2*s3,  s5*s4*s6*s3*s4*s2*s3,  s4*s6*s5*s4*s3*s2*s1,  s3*s4*s5*s2*s3*s4*s6,  s3*s4*s6*s2*s3*s4*s5,  s3*s4*s6*s5*s4*s3*s2,  s2*s3*s4*s6*s5*s4*s3,  s1*s2*s3*s4*s6*s5*s4,  s4*s6*s3*s4*s5*s2*s3*s4,  s4*s5*s3*s4*s6*s2*s3*s4,  s4*s6*s3*s4*s2*s3*s1*s2,  s4*s5*s3*s4*s2*s3*s1*s2,  s3*s4*s6*s5*s4*s3*s2*s1,  s2*s3*s4*s6*s1*s2*s3*s4,  s2*s3*s4*s5*s1*s2*s3*s4,  s2*s3*s4*s6*s5*s4*s3*s2,  s1*s2*s3*s4*s6*s5*s4*s3,  s6*s4*s5*s3*s4*s6*s2*s3*s4,  s6*s4*s5*s3*s4*s2*s3*s1*s2,  s5*s4*s6*s3*s4*s5*s2*s3*s4,  s5*s4*s6*s3*s4*s2*s3*s1*s2,  s4*s6*s3*s4*s5*s2*s3*s4*s6,  s4*s5*s3*s4*s6*s2*s3*s4*s5,  s4*s5*s3*s4*s6*s4*s5*s3*s4,  s3*s4*s6*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s2*s3*s4*s1*s2*s3,  s2*s3*s4*s5*s1*s2*s3*s4*s6,  s2*s3*s4*s6*s1*s2*s3*s4*s5,  s2*s3*s4*s6*s5*s4*s3*s2*s1,  s1*s2*s3*s4*s6*s5*s4*s3*s2,  s6*s4*s5*s3*s4*s6*s2*s3*s4*s5,  s5*s4*s6*s3*s4*s5*s2*s3*s4*s6,  s1*s2*s3*s4*s6*s5*s4*s3*s2*s1,  s4*s5*s3*s4*s6*s4*s5*s3*s4*s2*s3,  s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s4*s5*s3*s4*s6*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4,  s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4,  s6*s4*s5*s3*s4*s6*s2*s3*s4*s1*s2*s3,  s5*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6,  s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5,  s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4,  s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s4*s5*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3,  s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4,  s6*s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s5*s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4,  s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6,  s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5,  s6*s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6,  s5*s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5,  s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4,  s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3,  s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4,  s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4]

#dissectors: ?

In [ ]:
######################## B6 ########################

#sum_J KL = 146

#bigrassmannians: 161

bigrassmannians = [s3, s3*s2, s3*s4, s3*s2*s1, s3*s4*s5, s3*s4*s2*s3, s3*s4*s5*s6, s3*s4*s5*s6*s5, s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4, s3*s4*s5*s6*s5*s4, s3*s4*s5*s6*s5*s4*s3, s3*s4*s5*s2*s3*s4*s1*s2*s3, s2, s2*s3, s2*s3*s4, s2*s3*s4*s5, s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s5, s2*s3*s4*s5*s6*s5*s4, s2*s3*s4*s5*s6*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4, s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s5, s4*s5*s3*s4, s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s6, s4*s5*s6*s3*s4*s5, s4*s5*s6*s3*s4*s5*s6, s4*s5*s6*s3*s4*s5*s2*s3*s4, s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s6*s5, s4*s5*s6*s5*s3*s4, s4*s5*s6*s5*s3*s4*s2*s3, s4*s5*s6*s3*s4*s5*s6*s4*s5, s4*s5*s6*s5*s3*s4*s2*s3*s1*s2, s4*s5*s6*s5*s4, s4*s5*s6*s5*s4*s3, s4*s5*s6*s5*s4*s3*s2, s4*s5*s6*s5*s4*s3*s2*s1, s4*s5*s6*s3*s4*s5*s6*s4*s5*s3*s4, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s4*s5, s1*s2*s3*s4*s5*s6, s1*s2*s3*s4*s5*s6*s5, s1*s2*s3*s4*s5*s6*s5*s4, s1*s2*s3*s4*s5*s6*s5*s4*s3, s5, s5*s4, s5*s4*s3, s5*s4*s3*s2, s5*s4*s3*s2*s1, s5*s6, s5*s6*s4*s5, s5*s6*s4*s5*s3*s4, s5*s6*s4*s5*s3*s4*s2*s3, s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s5*s6*s4*s5*s6, s5*s6*s4*s5*s6*s3*s4*s5, s5*s6*s4*s5*s6*s3*s4*s5*s6, s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s6*s5, s5*s6*s5*s4, s5*s6*s5*s4*s3, s5*s6*s5*s4*s3*s2, s5*s6*s5*s4*s3*s2*s1, s5*s6*s4*s5*s6*s4*s5, s5*s6*s4*s5*s6*s4*s5*s3*s4, s5*s6*s4*s5*s6*s4*s5*s3*s4*s2*s3, s5*s6*s4*s5*s6*s3*s4*s5*s6*s3*s4*s5, s5*s6*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6, s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5, s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5*s3*s4, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s2*s3*s4*s5*s6*s5*s1*s2*s3*s4, s2*s3*s4*s5*s6*s5*s4*s1*s2*s3, s2*s3*s4*s5*s6*s5*s4*s3*s1*s2, s6, s6*s5, s6*s5*s4, s6*s5*s4*s3, s6*s5*s4*s3*s2, s6*s5*s4*s3*s2*s1, s6*s5*s6, s6*s5*s6*s4*s5, s6*s5*s6*s4*s5*s3*s4, s6*s5*s6*s4*s5*s3*s4*s2*s3, s6*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s5*s6*s4*s5*s6, s6*s5*s6*s4*s5*s6*s3*s4*s5, s6*s5*s6*s4*s5*s6*s3*s4*s5*s6, s6*s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4, s6*s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s5*s6*s5*s4*s3*s2*s1, s1*s2*s3*s4*s5*s6*s5*s4*s3*s2, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s2*s3*s4*s5, s5*s6*s4*s5*s6*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s4*s5*s6*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s2*s3*s4*s5, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s5*s4*s3*s2*s1, s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4*s1*s2*s3, s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s1*s2*s3*s4, s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4, s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s3*s4*s5, s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4, s4*s5*s6*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3, s4*s5*s6*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5, s4*s5*s6*s3*s4*s5*s6*s4*s5*s2*s3*s4, s4*s5*s6*s3*s4*s5*s6*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s5*s6*s5*s4*s3*s2, s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3, s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4, s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5, s3*s4*s5*s6*s2*s3*s4*s5*s6, s3*s4*s5*s6*s2*s3*s4*s5, s3*s4*s5*s6*s5*s2*s3*s4, s3*s4*s5*s6*s5*s4*s2*s3, s3*s4*s5*s6*s5*s4*s3*s2, s3*s4*s5*s6*s5*s4*s3*s2*s1, s3*s4*s5*s6*s5*s4*s2*s3*s1*s2, s3*s4*s5*s6*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4]

#join_irreducibles = dissectors: 146

join_irreducibles = [s6,  s5,  s4,  s3,  s2,  s1,  s6*s5,  s5*s6,  s5*s4,  s4*s5,  s4*s3,  s3*s4,  s3*s2,  s2*s3,  s2*s1,  s1*s2,  s6*s5*s6,  s6*s5*s4,  s5*s6*s5,  s5*s4*s3,  s4*s5*s6,  s4*s3*s2,  s3*s4*s5,  s3*s2*s1,  s2*s3*s4,  s1*s2*s3,  s6*s5*s4*s3,  s5*s6*s4*s5,  s5*s6*s5*s4,  s5*s4*s3*s2,  s4*s5*s6*s5,  s4*s5*s3*s4,  s4*s3*s2*s1,  s3*s4*s5*s6,  s3*s4*s2*s3,  s2*s3*s4*s5,  s2*s3*s1*s2,  s1*s2*s3*s4,  s6*s5*s6*s4*s5,  s6*s5*s4*s3*s2,  s5*s6*s4*s5*s6,  s5*s6*s5*s4*s3,  s5*s4*s3*s2*s1,  s4*s5*s6*s5*s4,  s3*s4*s5*s6*s5,  s2*s3*s4*s5*s6,  s1*s2*s3*s4*s5,  s6*s5*s6*s4*s5*s6,  s6*s5*s4*s3*s2*s1,  s5*s6*s4*s5*s3*s4,  s5*s6*s5*s4*s3*s2,  s4*s5*s6*s3*s4*s5,  s4*s5*s6*s5*s4*s3,  s4*s5*s3*s4*s2*s3,  s3*s4*s5*s6*s5*s4,  s3*s4*s5*s2*s3*s4,  s3*s4*s2*s3*s1*s2,  s2*s3*s4*s5*s6*s5,  s2*s3*s4*s1*s2*s3,  s1*s2*s3*s4*s5*s6,  s6*s5*s6*s4*s5*s3*s4,  s5*s6*s4*s5*s6*s4*s5,  s5*s6*s5*s4*s3*s2*s1,  s4*s5*s6*s3*s4*s5*s6,  s4*s5*s6*s5*s4*s3*s2,  s3*s4*s5*s6*s5*s4*s3,  s2*s3*s4*s5*s6*s5*s4,  s1*s2*s3*s4*s5*s6*s5,  s5*s6*s4*s5*s6*s3*s4*s5,  s5*s6*s4*s5*s3*s4*s2*s3,  s4*s5*s3*s4*s2*s3*s1*s2,  s4*s5*s6*s5*s4*s3*s2*s1,  s3*s4*s5*s6*s2*s3*s4*s5,  s3*s4*s5*s6*s5*s4*s3*s2,  s2*s3*s4*s5*s1*s2*s3*s4,  s2*s3*s4*s5*s6*s5*s4*s3,  s1*s2*s3*s4*s5*s6*s5*s4,  s6*s5*s6*s4*s5*s6*s3*s4*s5,  s6*s5*s6*s4*s5*s3*s4*s2*s3,  s5*s6*s4*s5*s6*s3*s4*s5*s6,  s5*s6*s4*s5*s6*s4*s5*s3*s4,  s4*s5*s6*s3*s4*s5*s6*s4*s5,  s4*s5*s6*s3*s4*s5*s2*s3*s4,  s3*s4*s5*s6*s2*s3*s4*s5*s6,  s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s6*s5*s4*s3*s2*s1,  s2*s3*s4*s5*s6*s5*s4*s3*s2,  s1*s2*s3*s4*s5*s6*s5*s4*s3,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s6,  s5*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s2*s3*s4*s5*s6*s1*s2*s3*s4*s5,  s2*s3*s4*s5*s6*s5*s4*s3*s2*s1,  s1*s2*s3*s4*s5*s6*s5*s4*s3*s2,  s6*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4,  s5*s6*s4*s5*s6*s4*s5*s3*s4*s2*s3,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5,  s4*s5*s6*s3*s4*s5*s6*s4*s5*s3*s4,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5,  s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6,  s1*s2*s3*s4*s5*s6*s5*s4*s3*s2*s1,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s3*s4*s5,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6,  s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5,  s5*s6*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s4*s5*s6*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4,  s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6,  s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s4*s5*s6*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3,  s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5*s3*s4,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s2*s3*s4*s5,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s3*s4*s5,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5,  s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s2*s3*s4*s5,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4]

dissectors = [x for x in join_irreducibles]


In [37]:
######################## E6 ########################

#sum_J KL = 156

#bigrassmannians: 232

bigrassmannians = [s6, s6*s5, s6*s5*s4, s6*s5*s4*s2, s6*s5*s4*s3, s6*s5*s4*s3*s1, s6*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5, s5*s6, s5*s4, s5*s6*s4*s5, s5*s4*s2, s5*s6*s4*s5*s2*s4, s5*s6*s4*s5*s2*s4*s3, s5*s6*s4*s5*s2*s4*s3*s1, s5*s4*s3, s5*s6*s4*s5*s3*s4, s5*s6*s4*s5*s3*s4*s2, s5*s6*s4*s5*s3*s2*s4, s5*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4, s5*s6*s4*s3*s2*s4*s5, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4, s4*s5, s4*s5*s6, s4*s2, s4*s5*s2*s4, s4*s5*s6*s2*s4*s5, s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3, s4*s5*s3*s4, s4*s5*s6*s3*s4*s5, s4*s5*s3*s4*s2, s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5, s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s1, s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3, s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4, s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2, s2*s4, s2*s4*s5, s2*s4*s5*s6, s2*s4*s3, s2*s4*s5*s3*s4, s2*s4*s5*s6*s3*s4*s5, s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s2*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3, s3*s4, s3*s4*s5, s3*s4*s5*s6, s3*s4*s2, s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s1, s3*s4*s5*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3, s3*s4*s5*s1*s3*s4, s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1, s1*s3, s1*s3*s4, s1*s3*s4*s5, s1*s3*s4*s5*s6, s1*s3*s4*s2, s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5, s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6]

#join_irreducibles: 182

join_irreducibles = [s1, s2, s3, s4, s5, s6, s1*s3, s2*s4, s3*s1, s3*s4, s4*s2, s4*s3, s4*s5, s5*s4, s5*s6, s6*s5, s1*s3*s4, s2*s4*s3, s2*s4*s5, s3*s4*s2, s3*s4*s5, s4*s3*s1, s4*s5*s6, s5*s4*s2, s5*s4*s3, s6*s5*s4, s1*s3*s4*s2, s1*s3*s4*s5, s2*s4*s3*s1, s2*s4*s5*s6, s3*s4*s1*s3, s3*s4*s5*s6, s4*s3*s2*s4, s4*s5*s2*s4, s4*s5*s3*s4, s5*s4*s3*s1, s5*s6*s4*s5, s6*s5*s4*s2, s6*s5*s4*s3, s1*s3*s4*s5*s6, s2*s4*s5*s3*s4, s3*s4*s5*s2*s4, s4*s5*s3*s4*s2, s4*s5*s2*s4*s3, s4*s3*s2*s4*s5, s5*s4*s3*s2*s4, s6*s5*s4*s3*s1, s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s2, s3*s4*s5*s2*s4*s3, s3*s4*s1*s3*s2*s4, s3*s4*s5*s1*s3*s4, s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5, s4*s5*s6*s3*s4*s5, s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4, s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4, s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s3*s4*s5, s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5, s5*s6*s4*s5*s3*s4*s2, s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4, s5*s6*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3, s6*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4]

#dissectors: ?

In [40]:
S = [4324561345243, 4563413245432413, 43245643245432413]

join_B(S)

432456413245432413

In [49]:
E = 43245432413
F = 4562453413
G = 4324561345243

In [54]:
print(join_B([E,G]))
print(join_B([F,G]))
print(join_B([E,F,G]))

432456134532413
432456134532413
432456134532413


In [48]:
join_B([E,F])

True